In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [2]:
apple = pd.read_csv("prices_and_sentiment_score.csv")[["Date", "score", "Adj Close"]]
apple.head()

,Date,score,Adj Close
0,2014-01-28,0.039278,16.087397
1,2014-01-29,0.026540,15.904763
2,2014-01-30,0.029652,15.873960
3,2014-01-31,0.213817,15.900002
4,2014-02-03,-0.028657,15.929543


In [3]:
#Splitting the dataset into training, validation, and testing dataset
#Train ratio = 64 %
#Validation ratio = 16%
#Test ratio = 20%

train_apple = apple[0:int(len(apple)*0.8)]
X_train_apple, X_validate_apple, test_apple = np.split(apple, [int(0.64 * len(apple)), int(0.8 * len(apple))])

# Setting Date column as an index, this will help to perform data visualization with x-axis as a Date column
train_apple = train_apple.set_index('Date', drop= False)
X_train_apple = X_train_apple.set_index('Date', drop= False)
X_validate_apple = X_validate_apple.set_index('Date', drop= False)
test_apple = test_apple.set_index('Date', drop= False)


print(len(X_train_apple), len(X_validate_apple), len(test_apple))

464 116 145


In [4]:
# Feature Scaling: We need data to be scaled as gradient descent based optimization algorithm requires it.
from sklearn.preprocessing import MinMaxScaler

# Feature scaling for apple stock closing price
stock_price_scaling_object = MinMaxScaler()

closing_price_train = X_train_apple['Adj Close'].values.reshape(-1, 1)
closing_price_validate = X_validate_apple['Adj Close'].values.reshape(-1, 1)
closing_price_test = test_apple['Adj Close'].values.reshape(-1, 1)

minmax_scaled_training_stock_price_data = stock_price_scaling_object.fit_transform(closing_price_train)
minmax_scaled_validation_stock_price_data = stock_price_scaling_object.transform(closing_price_validate)
minmax_scaled_test_stock_price_data = stock_price_scaling_object.transform(closing_price_test)

# Feature scaling for apple stock sentiment score
sentiment_score_scaling_object = MinMaxScaler()

sentiment_score_train = X_train_apple['score'].values.reshape(-1, 1)
sentiment_score_validate = X_validate_apple['score'].values.reshape(-1, 1)
sentiment_score_test = test_apple['score'].values.reshape(-1, 1)

minmax_scaled_training_sentiment_score_data = sentiment_score_scaling_object.fit_transform(sentiment_score_train)
minmax_scaled_validation_sentiment_score_data = sentiment_score_scaling_object.transform(sentiment_score_validate)
minmax_scaled_test_sentiment_score_data = sentiment_score_scaling_object.transform(sentiment_score_test)

In [5]:
# Preparing the dataset for training the model

# This is like a regression task, so we need two datasets the input (X) and the output(Y)


# The sequential relationship is maintained in the datasets, because we are currently exploiting the sequential model.
def data_preparation(stockPrice, sentimentScore, sequenceLength):
    
    # X is acting like an input feature vectors (combination of stock price + sentiment score of last day).
    # Y is acting like a vector of corresponding target (last day stock price) values.
    X, Y = [], []
    for i in range(len(stockPrice)-sequenceLength-1):
        
        # iteratively extracting the look back sequence length of stock price data. Here sequenceLength = 40
        stock_price = stockPrice[i:(i+sequenceLength), 0]
        
        # iteratively extracting the sentment score of last day
        sentiment_score = sentimentScore[i+sequenceLength-1]
        
        # X is a combination of the stock price of previous 40 days and sentiment score of the last day
        X.append(np.append(stock_price,sentiment_score))
        
        # Y is the stock price of the last day. This next value will be treated as the target
        Y.append(stockPrice[i + sequenceLength, 0])
        
    return np.array(X), np.array(Y)


train_input_featureX, train_targetY = data_preparation(minmax_scaled_training_stock_price_data, 
                                                       minmax_scaled_training_sentiment_score_data, 
                                                       40)

validation_input_featureX, validation_targetY = data_preparation(minmax_scaled_validation_stock_price_data, 
                                                                 minmax_scaled_validation_sentiment_score_data, 
                                                                 40)

test_input_featureX, test_targetY = data_preparation(minmax_scaled_test_stock_price_data, 
                                                     minmax_scaled_test_sentiment_score_data, 
                                                     40)





In [6]:
# Here, For example, train_input_featureX has 423 sequences and each sequences have total 41 time-steps.
# After this, we will transform these matrices into different batches by reshaping them so that 
# they get accepted by our recurrent neural network.

train_input_featureX.shape, validation_input_featureX.shape, test_input_featureX.shape

((423, 41), (75, 41), (104, 41))

In [7]:
#Converting the above prepared datasets into 3-dimentional tensor to handle the batch gradient descent.

train_input_featureX, train_targetY = np.array(train_input_featureX), np.array(train_targetY)
train_input_featureX = np.reshape(train_input_featureX, 
                                  (train_input_featureX.shape[0], train_input_featureX.shape[1], 1))

validation_input_featureX, validation_targetY = np.array(validation_input_featureX), np.array(validation_targetY)
validation_input_featureX = np.reshape(validation_input_featureX, 
                                       (validation_input_featureX.shape[0], validation_input_featureX.shape[1], 1))

test_input_featureX, test_targetY = np.array(test_input_featureX), np.array(test_targetY)
test_input_featureX = np.reshape(test_input_featureX, 
                                 (test_input_featureX.shape[0], test_input_featureX.shape[1], 1))

train_input_featureX.shape, validation_input_featureX.shape, test_input_featureX.shape

((423, 41, 1), (75, 41, 1), (104, 41, 1))

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend
from math import sqrt
from sklearn.metrics import mean_squared_error

In [ ]:
'''
LSTM RNN Hyperparameter Tuning.
This was adapted from a kaggle post on 05/08/2021.

Kaggle Post here: 
https://www.kaggle.com/kamyarazar/stock-price-prediction-lstm-hyperparameter-tuning

'''

In [9]:
from itertools import product
def LSTM_RNN_Hyperparameter_Optimization(hyper_parameter_value_matrix,
                                         train_input_featureX,
                                         train_targetY,
                                         validation_input_featureX,
                                         validation_targetY,
                                         test_input_featureX):
    
    # Unpacking all the hyperparamters arrays
    number_of_units_list, rate_of_learning_list, number_of_epochs_list, number_of_batches_list = hyper_parameter_value_matrix
    
    
    # Gernerating all possible combinations of hyperparameter values using itertool.product() function
    all_combinations_of_hyperparameter_values = list(product(number_of_units_list,
                                                             rate_of_learning_list, 
                                                             number_of_epochs_list,
                                                             number_of_batches_list))
    result = []
    
    for i in range(0, len(all_combinations_of_hyperparameter_values)):
        
        print(f'{i+1}th combination: \n')
        print("*****************************START**************************************")
        
        # tuple unpacking of hyperparameter values
        number_of_units, rate_of_learning, number_of_epochs, number_of_batches = all_combinations_of_hyperparameter_values[i]
        
    
        # Defining the sequential model
        model = Sequential()

        # Adding LSTM layer with 'number_of_units' units, hyperbolic tangent as an activation function,
        # and input shape in case of train_input_featureX will be (100, 1)
        model.add(LSTM(units=number_of_units, activation='tanh', 
                       kernel_initializer=tf.keras.initializers.glorot_uniform(seed=26), 
                       input_shape = (train_input_featureX.shape[1], 1), 
                       unroll = True))

        # After setting the recurrent layer, now adding the output layer
        model.add(Dense(1, name="output_layer"))


        #Defining our loss function
        def root_mean_squared_error(y_true, y_pred):

            error = y_pred - y_true
            squared_error = backend.square(error)
            mean_squared_error = backend.mean(squared_error)
            square_root_of_mean_squared_error = backend.sqrt(mean_squared_error)
            return square_root_of_mean_squared_error

        # RNN model compiling
        model.compile(optimizer = keras.optimizers.Adam(learning_rate=rate_of_learning), 
                      loss = root_mean_squared_error)



        # Fitting the RNN model to the Training set
        model.fit(train_input_featureX, train_targetY, 
                  epochs = number_of_epochs, 
                  batch_size = number_of_batches,  
                  validation_data = (validation_input_featureX, validation_targetY))
        
        
        #Predicting on test data
        predicted_stock_price = model.predict(test_input_featureX)
        predicted_stock_price = stock_price_scaling_object.inverse_transform(predicted_stock_price)
        
        # Root mean square error
        RMSE_LSTM = sqrt(mean_squared_error(test_apple.iloc[41:]['Adj Close'].values, predicted_stock_price))
        
        result.append(list((number_of_units, rate_of_learning, 
                            number_of_epochs, number_of_batches,
                            RMSE_LSTM)))
        print("*****************************END**************************************")

         
    return result
        

In [10]:
# Two dimensional matrix (Hyper Parameter value matrix)

# [
#     [units], 
#     [learning_rate],
#     [epochs],
#     [batch_size]
# ]

hyper_parameter_value_matrix = [
    
    [115, 120, 125, 130],
    [0.01, 0.03, 0.06, 0.09],
    [44, 46, 48, 50],
    [8, 12, 16, 32]
]

hyperparameter_tuning = LSTM_RNN_Hyperparameter_Optimization(hyper_parameter_value_matrix,
                                                             train_input_featureX,
                                                             train_targetY,
                                                             validation_input_featureX,
                                                             validation_targetY,
                                                             test_input_featureX)


1th combination: 

*****************************START**************************************
Epoch 1/44
53/53 [==============================] - 5s 22ms/step - loss: 0.1370 - val_loss: 0.0594
Epoch 2/44
53/53 [==============================] - 1s 10ms/step - loss: 0.0507 - val_loss: 0.0583
Epoch 3/44
53/53 [==============================] - 1s 11ms/step - loss: 0.0444 - val_loss: 0.0937
Epoch 4/44
53/53 [==============================] - 0s 9ms/step - loss: 0.0460 - val_loss: 0.0277
Epoch 5/44
53/53 [==============================] - 0s 9ms/step - loss: 0.0326 - val_loss: 0.0246
Epoch 6/44
53/53 [==============================] - 0s 9ms/step - loss: 0.0317 - val_loss: 0.0261
Epoch 7/44
53/53 [==============================] - 0s 9ms/step - loss: 0.0339 - val_loss: 0.0326
Epoch 8/44
53/53 [==============================] - 0s 9ms/step - loss: 0.0377 - val_loss: 0.0238
Epoch 9/44
53/53 [==============================] - 0s 9ms/step - loss: 0.0345 - val_loss: 0.0246
Epoch 10/44
53/53 [====

Epoch 27/44
14/14 [==============================] - 0s 30ms/step - loss: 0.0329 - val_loss: 0.0343
Epoch 28/44
14/14 [==============================] - 0s 29ms/step - loss: 0.0320 - val_loss: 0.0290
Epoch 29/44
14/14 [==============================] - 0s 29ms/step - loss: 0.0413 - val_loss: 0.0305
Epoch 30/44
14/14 [==============================] - 0s 28ms/step - loss: 0.0400 - val_loss: 0.0561
Epoch 31/44
14/14 [==============================] - 0s 30ms/step - loss: 0.0422 - val_loss: 0.0401
Epoch 32/44
14/14 [==============================] - 0s 32ms/step - loss: 0.0351 - val_loss: 0.0244
Epoch 33/44
14/14 [==============================] - 1s 38ms/step - loss: 0.0362 - val_loss: 0.0287
Epoch 34/44
14/14 [==============================] - 0s 30ms/step - loss: 0.0338 - val_loss: 0.0246
Epoch 35/44
14/14 [==============================] - 0s 29ms/step - loss: 0.0279 - val_loss: 0.0290
Epoch 36/44
14/14 [==============================] - 0s 30ms/step - loss: 0.0300 - val_loss: 0.0299


Epoch 16/46
36/36 [==============================] - 1s 16ms/step - loss: 0.0296 - val_loss: 0.0358
Epoch 17/46
36/36 [==============================] - 1s 15ms/step - loss: 0.0332 - val_loss: 0.0250
Epoch 18/46
36/36 [==============================] - 1s 15ms/step - loss: 0.0285 - val_loss: 0.0251
Epoch 19/46
36/36 [==============================] - 1s 15ms/step - loss: 0.0321 - val_loss: 0.0245
Epoch 20/46
36/36 [==============================] - 1s 16ms/step - loss: 0.0338 - val_loss: 0.0234
Epoch 21/46
36/36 [==============================] - 1s 16ms/step - loss: 0.0409 - val_loss: 0.0254
Epoch 22/46
36/36 [==============================] - 1s 15ms/step - loss: 0.0284 - val_loss: 0.0263
Epoch 23/46
36/36 [==============================] - 1s 15ms/step - loss: 0.0296 - val_loss: 0.0265
Epoch 24/46
36/36 [==============================] - 1s 15ms/step - loss: 0.0315 - val_loss: 0.0239
Epoch 25/46
36/36 [==============================] - 1s 15ms/step - loss: 0.0303 - val_loss: 0.0238


Epoch 3/46
14/14 [==============================] - 0s 32ms/step - loss: 0.0632 - val_loss: 0.0396
Epoch 4/46
14/14 [==============================] - 0s 32ms/step - loss: 0.0437 - val_loss: 0.0505
Epoch 5/46
14/14 [==============================] - 0s 32ms/step - loss: 0.0417 - val_loss: 0.0386
Epoch 6/46
14/14 [==============================] - 0s 32ms/step - loss: 0.0378 - val_loss: 0.0381
Epoch 7/46
14/14 [==============================] - 0s 32ms/step - loss: 0.0371 - val_loss: 0.0338
Epoch 8/46
14/14 [==============================] - 0s 31ms/step - loss: 0.0406 - val_loss: 0.0326
Epoch 9/46
14/14 [==============================] - 0s 30ms/step - loss: 0.0332 - val_loss: 0.0333
Epoch 10/46
14/14 [==============================] - 0s 30ms/step - loss: 0.0324 - val_loss: 0.0330
Epoch 11/46
14/14 [==============================] - 0s 31ms/step - loss: 0.0348 - val_loss: 0.0298
Epoch 12/46
14/14 [==============================] - 0s 31ms/step - loss: 0.0336 - val_loss: 0.0348
Epoch 1

Epoch 20/48
27/27 [==============================] - 0s 15ms/step - loss: 0.0321 - val_loss: 0.0253
Epoch 21/48
27/27 [==============================] - 0s 15ms/step - loss: 0.0285 - val_loss: 0.0248
Epoch 22/48
27/27 [==============================] - 0s 15ms/step - loss: 0.0288 - val_loss: 0.0268
Epoch 23/48
27/27 [==============================] - 0s 15ms/step - loss: 0.0271 - val_loss: 0.0281
Epoch 24/48
27/27 [==============================] - 0s 15ms/step - loss: 0.0298 - val_loss: 0.0266
Epoch 25/48
27/27 [==============================] - 0s 15ms/step - loss: 0.0332 - val_loss: 0.0260
Epoch 26/48
27/27 [==============================] - 0s 15ms/step - loss: 0.0299 - val_loss: 0.0243
Epoch 27/48
27/27 [==============================] - 0s 16ms/step - loss: 0.0337 - val_loss: 0.0390
Epoch 28/48
27/27 [==============================] - 0s 15ms/step - loss: 0.0342 - val_loss: 0.0276
Epoch 29/48
27/27 [==============================] - 0s 16ms/step - loss: 0.0281 - val_loss: 0.0245


...

Epoch 35/46
53/53 [==============================] - 1s 15ms/step - loss: 0.2335 - val_loss: 0.2853
Epoch 36/46
53/53 [==============================] - 1s 16ms/step - loss: 0.1753 - val_loss: 0.2588
Epoch 37/46
53/53 [==============================] - 1s 16ms/step - loss: 0.3135 - val_loss: 0.2005
Epoch 38/46
53/53 [==============================] - 1s 15ms/step - loss: 0.1882 - val_loss: 0.1054
Epoch 39/46
53/53 [==============================] - 1s 16ms/step - loss: 0.1409 - val_loss: 0.1378
Epoch 40/46
53/53 [==============================] - 1s 16ms/step - loss: 0.1540 - val_loss: 0.2326
Epoch 41/46
53/53 [==============================] - 1s 16ms/step - loss: 0.1935 - val_loss: 0.1746
Epoch 42/46
53/53 [==============================] - 1s 16ms/step - loss: 0.1850 - val_loss: 0.1153
Epoch 43/46
53/53 [==============================] - 1s 16ms/step - loss: 0.1777 - val_loss: 0.1839
Epoch 44/46
53/53 [==============================] - 1s 16ms/step - loss: 0.2895 - val_loss: 0.3049


Epoch 22/46
27/27 [==============================] - 1s 22ms/step - loss: 0.1574 - val_loss: 0.1421
Epoch 23/46
27/27 [==============================] - 1s 23ms/step - loss: 0.1340 - val_loss: 0.1322
Epoch 24/46
27/27 [==============================] - 1s 23ms/step - loss: 0.1373 - val_loss: 0.1027
Epoch 25/46
27/27 [==============================] - 1s 23ms/step - loss: 0.1345 - val_loss: 0.1080
Epoch 26/46
27/27 [==============================] - 1s 22ms/step - loss: 0.1259 - val_loss: 0.1676
Epoch 27/46
27/27 [==============================] - 1s 23ms/step - loss: 0.1280 - val_loss: 0.1004
Epoch 28/46
27/27 [==============================] - 1s 23ms/step - loss: 0.1153 - val_loss: 0.1048
Epoch 29/46
27/27 [==============================] - 1s 22ms/step - loss: 0.1150 - val_loss: 0.1006
Epoch 30/46
27/27 [==============================] - 1s 22ms/step - loss: 0.1437 - val_loss: 0.2320
Epoch 31/46
27/27 [==============================] - 1s 22ms/step - loss: 0.1992 - val_loss: 0.1669


Epoch 9/48
53/53 [==============================] - 1s 15ms/step - loss: 0.3679 - val_loss: 0.4399
Epoch 10/48
53/53 [==============================] - 1s 15ms/step - loss: 0.4954 - val_loss: 0.4773
Epoch 11/48
53/53 [==============================] - 1s 15ms/step - loss: 0.4366 - val_loss: 0.1729
Epoch 12/48
53/53 [==============================] - 1s 15ms/step - loss: 0.3246 - val_loss: 0.1283
Epoch 13/48
53/53 [==============================] - 1s 15ms/step - loss: 0.3681 - val_loss: 0.1239
Epoch 14/48
53/53 [==============================] - 1s 15ms/step - loss: 0.4814 - val_loss: 0.6168
Epoch 15/48
53/53 [==============================] - 1s 15ms/step - loss: 0.4651 - val_loss: 0.2383
Epoch 16/48
53/53 [==============================] - 1s 15ms/step - loss: 0.5606 - val_loss: 0.5401
Epoch 17/48
53/53 [==============================] - 1s 15ms/step - loss: 0.4070 - val_loss: 0.3301
Epoch 18/48
53/53 [==============================] - 1s 15ms/step - loss: 0.5044 - val_loss: 0.3856
E

Epoch 24/48
14/14 [==============================] - 0s 33ms/step - loss: 0.2675 - val_loss: 0.3504
Epoch 25/48
14/14 [==============================] - 0s 32ms/step - loss: 0.2622 - val_loss: 0.2194
Epoch 26/48
14/14 [==============================] - 0s 32ms/step - loss: 0.2701 - val_loss: 0.3336
Epoch 27/48
14/14 [==============================] - 0s 32ms/step - loss: 0.2620 - val_loss: 0.2416
Epoch 28/48
14/14 [==============================] - 0s 34ms/step - loss: 0.2682 - val_loss: 0.3549
Epoch 29/48
14/14 [==============================] - 0s 32ms/step - loss: 0.2616 - val_loss: 0.2578
Epoch 30/48
14/14 [==============================] - 0s 34ms/step - loss: 0.2704 - val_loss: 0.3725
Epoch 31/48
14/14 [==============================] - 0s 33ms/step - loss: 0.2583 - val_loss: 0.2106
Epoch 32/48
14/14 [==============================] - 0s 33ms/step - loss: 0.2549 - val_loss: 0.4145
Epoch 33/48
14/14 [==============================] - 0s 33ms/step - loss: 0.2636 - val_loss: 0.1881


Epoch 5/50
36/36 [==============================] - 1s 17ms/step - loss: 0.3085 - val_loss: 0.6505
Epoch 6/50
36/36 [==============================] - 1s 17ms/step - loss: 0.3578 - val_loss: 0.0948
Epoch 7/50
36/36 [==============================] - 1s 17ms/step - loss: 0.2847 - val_loss: 0.1097
Epoch 8/50
36/36 [==============================] - 1s 18ms/step - loss: 0.3393 - val_loss: 0.4187
Epoch 9/50
36/36 [==============================] - 1s 17ms/step - loss: 0.3536 - val_loss: 0.4508
Epoch 10/50
36/36 [==============================] - 1s 17ms/step - loss: 0.3308 - val_loss: 0.1734
Epoch 11/50
36/36 [==============================] - 1s 17ms/step - loss: 0.3205 - val_loss: 0.3110
Epoch 12/50
36/36 [==============================] - 1s 18ms/step - loss: 0.3594 - val_loss: 0.4861
Epoch 13/50
36/36 [==============================] - 1s 17ms/step - loss: 0.2788 - val_loss: 0.2041
Epoch 14/50
36/36 [==============================] - 1s 18ms/step - loss: 0.2675 - val_loss: 0.1372
Epoch

In [15]:
hyperparameter_tuning = pd.DataFrame(hyperparameter_tuning)
hyperparameter_tuning = hyperparameter_tuning.sort_values(by=[4], ascending=True)

# Top 5 hyperparameters values
hyperparameter_tuning.head()

,0,1,2,3,4
73,120,0.01,48,12,0.422099
76,120,0.01,50,8,0.424579
68,120,0.01,46,8,0.425197
128,125,0.01,44,8,0.427146
193,130,0.01,44,12,0.427801


##### As we can clearly see after the hyperparameter tuning that the top row have the best value for our hyperparameters that yields the RMSE of 0.422099, which is slighlty better than our previous RMSE value of 0.43

##### Best Model Hyperparameters are: 
##### units = 120
##### learning_rate = 0.01
##### epochs = 48
##### batch_size = 12
